Séquentialisation des textes

Importation du fichier .csv adapté pour le nombre de fichier avec lesquels on travail

In [7]:
import os
import pandas as pd

# Charger le CSV
df = pd.read_csv('../data/training_set_metadata.csv', sep=';')
folder_path = "../data/digraphs"

# Vérifier les fichiers JSON
missing_files = []
for idx, row in df.iterrows():
    json_path = os.path.join(folder_path, f"{row['name']}.json")
    if not os.path.exists(json_path):
        missing_files.append(row['name'])

# Afficher les fichiers manquants
if missing_files:
    print("Fichiers manquants :")
    for file in missing_files:
        print(f"- {file}")
else:
    print("Tous les fichiers sont présents.")

# Optionnel : Mettre à jour le CSV pour ne conserver que les fichiers existants
if missing_files:
    df_updated = df[~df['name'].isin(missing_files)]
    df_updated.to_csv("../data/your_data_updated.csv", index=False)
    print("Fichier CSV mis à jour pour ne conserver que les fichiers existants.")

Fichiers manquants :
- ...
Fichier CSV mis à jour pour ne conserver que les fichiers existants.


Extraction du texte + vectorisation

In [9]:
import os
import re
import networkx as nx
import numpy as np
import spacy
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy.sparse import vstack, save_npz, load_npz
import gc
from tqdm import tqdm

# Expressions régulières pour extraire arêtes et nœuds
EDGE_REGEX = re.compile(r'"([a-zA-Z0-9]+)"\s*->\s*"([a-zA-Z0-9]+)"')
NODE_REGEX = re.compile(r'"([a-zA-Z0-9]+)"\s*\[label\s*=\s*"(.*?)"\s*\]')

def parse_cfg_file(cfg_path):
    try:
        with open(cfg_path, "r", encoding="utf-8", errors="ignore") as f:
            return [line.strip() for line in f if line.strip()]
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {cfg_path}: {e}")
        return []

def process_file(name, directory):
    """
    Lit un fichier JSON, construit un graphe avec NetworkX et extrait les instructions via DFS.
    Retourne une liste d'instructions (chaînes de caractères) ou None en cas d'erreur.
    """
    cfg_path = os.path.join(directory, f"{name}.json")
    if not os.path.exists(cfg_path):
        print(f"Le fichier {cfg_path} n'existe pas.")
        return None
    try:
        G = nx.DiGraph()
        lines = parse_cfg_file(cfg_path)
        if not lines:
            print(f"Le fichier {cfg_path} est vide ou n'a pas pu être lu.")
            return None

        # Extraction des arêtes et des nœuds
        for line in lines:
            if '->' in line:
                edge_match = EDGE_REGEX.match(line)
                if edge_match:
                    src, dst = edge_match.groups()
                    G.add_edge(src, dst)
            if "[" in line and "]" in line:
                node_match = NODE_REGEX.match(line)
                if node_match:
                    node, label = node_match.groups()
                    G.add_node(node, label=label)

        # Extraction des instructions via DFS
        instructions = []
        for node in nx.dfs_preorder_nodes(G):
            if 'label' in G.nodes[node]:
                instructions.append(G.nodes[node]['label'])
        print(f"Fichier {name}.json: {G.number_of_nodes()} nœuds, {G.number_of_edges()} arêtes.")
        return instructions

    except Exception as e:
        print(f"Erreur lors du traitement de {cfg_path}: {e}")
        return None

def batched(iterable, batch_size):
    """Générateur qui découpe un itérable en lots de taille batch_size."""
    batch = []
    for item in iterable:
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

def process_graphs_in_directory(directory, batch_size=100):
    output_dir = "../data/npz_matrices"
    files = [f for f in os.listdir(directory) if f.endswith(".json")]
    partial_files = []  # Pour enregistrer le nom des fichiers de batch
    batch_index = 0

    # Chargement du modèle SpaCy (si besoin pour d'autres traitements)
    nlp = spacy.load('en_core_web_sm')

    # Traitement par lots avec barre de progression
    for batch_files in tqdm(list(batched(files, batch_size)), desc="Traitement par batch"):
        docs = []  # Liste des documents (texte brut) du batch
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            futures = {executor.submit(process_file, os.path.splitext(filename)[0], directory): filename 
                       for filename in batch_files}
            for future in as_completed(futures):
                try:
                    result = future.result()
                    if result:
                        docs.append(" ".join(result))
                except Exception as e:
                    print(f"Erreur lors du traitement d'un fichier: {e}")
        gc.collect()

        if docs:
            # Utilisation du HashingVectorizer pour obtenir une matrice TF du batch
            hv = HashingVectorizer(n_features=2**20, alternate_sign=False, norm=None)
            batch_matrix = hv.transform(docs)
            # Enregistrement du batch sur disque
            batch_file = os.path.join(output_dir, f"tf_batch_{batch_index}.npz")
            save_npz(batch_file, batch_matrix)
            partial_files.append(batch_file)
            print(f"Batch {batch_index} enregistré : forme {batch_matrix.shape}")
            batch_index += 1

    # Si au moins un batch a été traité, on combine toutes les matrices
    if partial_files:
        matrices = [load_npz(f) for f in partial_files]
        full_tf_matrix = vstack(matrices)
        transformer = TfidfTransformer()
        tfidf_matrix = transformer.fit_transform(full_tf_matrix)
        final_file = os.path.join(output_dir, "tfidf_matrix.npz")
        save_npz(final_file, tfidf_matrix)
        print("Matrice TF-IDF finale de forme :", tfidf_matrix.shape)
    else:
        print("Aucun document traité.")

if __name__ == '__main__':
    directory = "../data/digraphs"
    process_graphs_in_directory(directory, batch_size=100)













Traitement par batch:   0%|          | 0/1 [00:00<?, ?it/s]

Fichier 0ad1f7ed3d2ed81ce7e68ce0ccb249399e5277bc094f7bc056941789f57ceb6a.json: 180 nœuds, 208 arêtes.
Fichier 0a7b1cd9ddae54ef50882b41754fbcba30361dd318601dca137bbebf635e8612.json: 1353 nœuds, 1468 arêtes.
Fichier 0a9e739910578192e2ad3f8f805a863d5cbcacbc38cde6488d7bbef5395dba87.json: 2822 nœuds, 4833 arêtes.
Fichier 00a20a5799aa786e2711b43682cf9d533719d5e404c2c5ef1709ac8fdec585f1.json: 443 nœuds, 508 arêtes.
Fichier 0a7f54525b7baab792fbf9160bc27f3b6424b955dae0b50248d14e41e362dbc0.json: 2721 nœuds, 3012 arêtes.
Fichier 0a49ffc1218b6ecfc0a03d115e24fda92e74a09f2bb0ac0d3b472068aa7eb3d4.json: 1763 nœuds, 2749 arêtes.
Fichier 0a5d1450c71befec5a5aeacaffc6dd7d6514de1693071af6c0b7ed1e2f9f540f.json: 1761 nœuds, 2799 arêtes.
Fichier 0a36a2c267bba80fb1f40b56aae0d84bfc5fcdc48b17978783bd7cc6045bc53a.json: 749 nœuds, 1127 arêtes.
Fichier 0acb1a3de29358883d838990adc60202faeda86bf7be1fdbf8fd3ad77d5a8877.json: 2757 nœuds, 3177 arêtes.
Fichier 0ac7a3df8ddc6e6b5670c1748dfdd80967131f511aaecaeef1c689efe2695

Traitement par batch: 100%|██████████| 1/1 [00:14<00:00, 14.93s/it]

Batch 0 enregistré : forme (50, 1048576)


Matrice TF-IDF finale de forme : (50, 1048576)


Modèle de prédiction

Modele MLP

Chargement et Prétraitement des Données

In [10]:
import pandas as pd
import numpy as np
from scipy.sparse import issparse, load_npz, csr_matrix, save_npz
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

def charger_donnees(csv_path='../data/your_data_updated.csv', matrix_path='../data/npz_matrices/tfidf_matrix.npz', variance_threshold=0.90):
    """Charge les données depuis un CSV et une matrice TF-IDF, puis applique Truncated SVD."""
    try:
        # Charger le fichier CSV
        df = pd.read_csv(csv_path, sep=';')

        # Charger la matrice TF-IDF
        X = load_npz(matrix_path)
        print("Matrice TF-IDF chargée de forme :", X.shape)

        # S'assurer que X est bien une matrice creuse
        if not issparse(X):
            X = csr_matrix(X)

        # Réduction initiale avec un nombre de composants fixe (max 300 ou nb de colonnes)
        svd = TruncatedSVD(n_components=min(X.shape[1], 300))
        X_reduced = svd.fit_transform(X)
        explained_variance = np.cumsum(svd.explained_variance_ratio_)
        print("Variance expliquée pour les 10 premières dimensions :", explained_variance[:10])

        # Déterminer le nombre optimal de composants pour capturer au moins 'variance_threshold' de variance
        n_components = np.argmax(explained_variance >= variance_threshold) + 1
        print(f"Nombre de composants pour capturer {variance_threshold*100}% de la variance : {n_components}")

        # Réduction définitive avec le nombre optimal de composants
        svd = TruncatedSVD(n_components=n_components)
        X_reduced = svd.fit_transform(X)
        print("Matrice TF-IDF réduite de forme :", X_reduced.shape)

        # Enregistrer la matrice réduite
        save_npz('../data/npz_matrices/reduced_tfidf_matrix.npz', csr_matrix(X_reduced))

        # Extraire les colonnes cibles (en excluant 'name')
        colonnes_cibles = df.columns.difference(['name'])
        y = df[colonnes_cibles].values

        # Binariser les cibles multi-labels
        mlb = MultiLabelBinarizer()
        y_binarized = mlb.fit_transform(y)
        print("Cibles binarisées de forme :", y_binarized.shape)

        # Aligner le nombre de lignes entre X et y
        if X_reduced.shape[0] != y_binarized.shape[0]:
            diff = X_reduced.shape[0] - y_binarized.shape[0]
            print(f"Alignement des données : Suppression de {diff} lignes de la matrice TF-IDF.")
            X_reduced = X_reduced[:y_binarized.shape[0]]

        # Retourner les données en divisant en ensembles d'entraînement et de test
        return train_test_split(X_reduced, y_binarized, test_size=0.2, random_state=42)

    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")
        return None

# Exemple d'utilisation
donnees = charger_donnees()
if donnees:
    X_train, X_test, y_train, y_test = donnees
    print("Données chargées et traitées avec succès.")
else:
    print("Échec du chargement des données.")



Matrice TF-IDF chargée de forme : (50, 1048576)


: 

Création du Modèle MLP

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.sparse import load_npz, issparse, csr_matrix, save_npz
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# -------------------------
# 1. Fonction de perte Focal Loss
# -------------------------
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-7
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return focal_loss_fixed

# -------------------------
# 2. Analyse de la distribution des labels
# -------------------------
def compute_label_distribution(y):
    label_freq = np.sum(y, axis=0)
    print("Distribution des labels (occurrences par label) :", label_freq)
    print("Pourcentage moyen d'occurrences par label :", np.mean(label_freq / y.shape[0]))
    return label_freq

# -------------------------
# 3. Chargement et prétraitement des données
#    avec filtrage des colonnes peu diversifiées
# -------------------------
def charger_donnees(csv_path='../data/your_data_updated.csv',
                    reduced_matrix_path='../data/npz_matrices/reduced_tfidf_matrix.npz',
                    apply_svd=False,
                    variance_threshold=0.9,
                    min_diversity=0.05):
    """
    Charge le CSV et la matrice TF-IDF.
    Les cibles sont toutes les colonnes du CSV sauf 'name'.
    Filtre également les colonnes peu diversifiées.
    """
    try:
        # Charger le CSV
        df = pd.read_csv(csv_path, sep=',')
        print("CSV chargé avec succès. Forme :", df.shape)
        print("Colonnes :", df.columns.tolist())

        # Charger la matrice TF-IDF (ou la matrice réduite)
        X = load_npz(reduced_matrix_path)
        print("Matrice TF-IDF chargée de forme :", X.shape)
        if not issparse(X):
            X = csr_matrix(X)

        # Si demandé, appliquer SVD pour réduire la dimension
        if apply_svd:
            svd = TruncatedSVD(n_components=min(X.shape[1], 300))
            X_temp = svd.fit_transform(X)
            explained_variance = np.cumsum(svd.explained_variance_ratio_)
            n_components = np.argmax(explained_variance >= variance_threshold) + 1
            print(f"Réduction SVD: {n_components} composantes pour capturer {variance_threshold*100}% de la variance.")
            svd = TruncatedSVD(n_components=n_components)
            X_reduced = svd.fit_transform(X)
        else:
            X_reduced = X

        # Extraire les cibles sous forme de DataFrame : toutes les colonnes sauf 'name'
        if 'name' not in df.columns:
            raise ValueError("La colonne 'name' est introuvable dans le CSV.")
        y_df = df.drop(columns=['name'])
        print("Cibles extraites de forme :", y_df.shape)

        # Filtrage des colonnes peu diversifiées
        # Calcul du pourcentage de 1 par colonne
        proportions = y_df.sum() / y_df.shape[0]
        # Conserver uniquement les colonnes dont le pourcentage de 1 est compris entre min_diversity et (1 - min_diversity)
        mask = (proportions >= min_diversity) & (proportions <= (1 - min_diversity))
        print(f"Nombre de colonnes avant filtrage diversité : {y_df.shape[1]}")
        y_df = y_df.loc[:, mask]
        print(f"Nombre de colonnes après filtrage diversité (seuil {min_diversity*100}%): {y_df.shape[1]}")

        y = y_df.values

        # Affichage de la distribution des labels après filtrage
        label_freq = np.sum(y, axis=0)
        print("Distribution des labels (après filtrage diversité) :", label_freq)
        print("Pourcentage moyen d'occurrences par label :", np.mean(label_freq / y.shape[0]))

        # Aligner les données si nécessaire
        if X_reduced.shape[0] != y.shape[0]:
            diff = X_reduced.shape[0] - y.shape[0]
            print(f"Alignement: suppression de {diff} lignes de la matrice TF-IDF.")
            X_reduced = X_reduced[:y.shape[0]]

        # Retourner les données divisées en ensembles d'entraînement et de test
        return train_test_split(X_reduced, y, test_size=0.2, random_state=42)

    except Exception as e:
        print("Erreur lors du chargement des données :", e)
        return None

# -------------------------
# Sélection des caractéristiques importantes
# -------------------------
def select_important_features(X_train, y_train, method='tree', threshold=None):
    if method == 'tree':
        # Utilisation d'un RandomForest pour sélectionner les caractéristiques
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        model = SelectFromModel(clf, prefit=True, threshold="mean")
    elif method == 'l1':
        # Utilisation de la régularisation L1 pour sélectionner les caractéristiques
        clf = LogisticRegression(penalty='l1', solver='saga', random_state=42, max_iter=1000)
        clf.fit(X_train, y_train)
        model = SelectFromModel(clf, prefit=True, threshold=threshold)
    else:
        raise ValueError("Méthode non supportée. Choisissez 'tree' ou 'l1'.")

    X_train_selected = model.transform(X_train)
    print(f"Nombre de caractéristiques sélectionnées : {X_train_selected.shape[1]}")
    return model

# -------------------------
# 4. Création du modèle MLP optimisé
# -------------------------
def creer_modele_mlp_optimise(input_dim, output_dim, use_focal_loss=False):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))

    # Architecture du modèle
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(output_dim, activation='sigmoid'))

    if use_focal_loss:
        loss_fn = focal_loss(gamma=2., alpha=0.25)
    else:
        loss_fn = 'binary_crossentropy'

    model.compile(optimizer=Adam(learning_rate=0.0005), loss=loss_fn, metrics=['accuracy'])
    return model

# -------------------------
# 5. Visualisation des résultats
# -------------------------
def visualiser_resultats(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title("Évolution de la loss pendant l'entraînement")
    plt.legend()
    plt.show()

# -------------------------
# 6. Entraînement et évaluation du modèle
# -------------------------
def entrainer_et_evaluer_modele(X_train, y_train, X_test, y_test, use_focal_loss=False):
    input_dim = X_train.shape[1]
    output_dim = y_train.shape[1]
    print("Dimension d'entrée :", input_dim, "Dimension de sortie :", output_dim)

    # Création du modèle
    modele = creer_modele_mlp_optimise(input_dim, output_dim, use_focal_loss=use_focal_loss)

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = modele.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2,
                         callbacks=[early_stopping], verbose=1)

    visualiser_resultats(history)

    y_pred = (modele.predict(X_test) > 0.5).astype("int32")
    score = f1_score(y_test, y_pred, average='macro', zero_division=0)
    print("Score F1 macro :", score)

    keras_model_path = "modele_mlp.h5"
    modele.save(keras_model_path)
    print(f"Modèle Keras sauvegardé sous {keras_model_path}")

    return modele

# -------------------------
# Exemple d'utilisation
# -------------------------
donnees = charger_donnees(csv_path='../data/your_data_updated.csv',
                          reduced_matrix_path='../data/npz_matrices/reduced_tfidf_matrix.npz',
                          apply_svd=False,
                          variance_threshold=0.90,
                          min_diversity=0.05)

if donnees:
    X_train, X_test, y_train, y_test = donnees

    # Sélection des caractéristiques importantes
    selector = select_important_features(X_train, y_train, method='tree')
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)

    # Entraînement et évaluation du modèle avec les caractéristiques sélectionnées
    modele = entrainer_et_evaluer_modele(X_train_selected, y_train, X_test_selected, y_test, use_focal_loss=True)
else:
    print("Échec du chargement des données.")

CSV chargé avec succès. Forme : (49, 454)
Colonnes : ['name', '64-bit execution via heavens gate', '64bits', 'PEB access', 'accept command line arguments', 'access the Windows event log', 'act as TCP client', 'allocate RW memory', 'allocate RWX memory', 'allocate memory', 'allocate or change RW memory', 'allocate or change RWX memory', 'allocate thread local storage', 'android', 'anorganix', 'apatch', 'apk', 'arm', 'armadillo', 'aspack', 'asprotect', 'assembly', 'attach user process memory', 'attachment', 'authenticate HMAC', 'block operations on executable memory pages using Arbitrary Code Guard', 'bobsoft', 'bypass Windows File Protection', 'calculate modulo 256 via x86 assembly', 'calls-wmi', 'capture screenshot', 'capture screenshot in Go', 'cexe', 'change file permission on Linux', 'change memory protection', 'change the wallpaper', 'check HTTP status code', 'check OS version', 'check ProcessDebugFlags', 'check ProcessDebugPort', 'check SystemKernelDebuggerInformation', 'check for

Prédiction sur les données de tests

Applications des transformations aux données tests

In [4]:
import os
import re
import networkx as nx
import numpy as np
import spacy
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from concurrent.futures import ThreadPoolExecutor, as_completed
from scipy.sparse import vstack, save_npz, load_npz
import gc
from tqdm import tqdm

# Expressions régulières pour extraire arêtes et nœuds
EDGE_REGEX = re.compile(r'"([a-zA-Z0-9]+)"\s*->\s*"([a-zA-Z0-9]+)"')
NODE_REGEX = re.compile(r'"([a-zA-Z0-9]+)"\s*\[label\s*=\s*"(.*?)"\s*\]')

def parse_cfg_file(cfg_path):
    try:
        with open(cfg_path, "r", encoding="utf-8", errors="ignore") as f:
            return [line.strip() for line in f if line.strip()]
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {cfg_path}: {e}")
        return []

def process_file(name, directory):
    """
    Lit un fichier JSON, construit un graphe avec NetworkX et extrait les instructions via DFS.
    Retourne une liste d'instructions (chaînes de caractères) ou None en cas d'erreur.
    """
    cfg_path = os.path.join(directory, f"{name}.json")
    if not os.path.exists(cfg_path):
        print(f"Le fichier {cfg_path} n'existe pas.")
        return None
    try:
        G = nx.DiGraph()
        lines = parse_cfg_file(cfg_path)
        if not lines:
            print(f"Le fichier {cfg_path} est vide ou n'a pas pu être lu.")
            return None

        # Extraction des arêtes et des nœuds
        for line in lines:
            if '->' in line:
                edge_match = EDGE_REGEX.match(line)
                if edge_match:
                    src, dst = edge_match.groups()
                    G.add_edge(src, dst)
            if "[" in line and "]" in line:
                node_match = NODE_REGEX.match(line)
                if node_match:
                    node, label = node_match.groups()
                    G.add_node(node, label=label)

        # Extraction des instructions via DFS
        instructions = []
        for node in nx.dfs_preorder_nodes(G):
            if 'label' in G.nodes[node]:
                instructions.append(G.nodes[node]['label'])
        print(f"Fichier {name}.json: {G.number_of_nodes()} nœuds, {G.number_of_edges()} arêtes.")
        return instructions

    except Exception as e:
        print(f"Erreur lors du traitement de {cfg_path}: {e}")
        return None

def batched(iterable, batch_size):
    """Générateur qui découpe un itérable en lots de taille batch_size."""
    batch = []
    for item in iterable:
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

def process_graphs_in_directory(directory, batch_size=100):
    files = [f for f in os.listdir(directory) if f.endswith(".json")]
    partial_files = []  # Pour enregistrer le nom des fichiers de batch
    batch_index = 0

    # Chargement du modèle SpaCy (si besoin pour d'autres traitements)
    nlp = spacy.load('en_core_web_sm')

    # Traitement par lots avec barre de progression
    for batch_files in tqdm(list(batched(files, batch_size)), desc="Traitement par batch"):
        docs = []  # Liste des documents (texte brut) du batch
        with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
            futures = {executor.submit(process_file, os.path.splitext(filename)[0], directory): filename 
                       for filename in batch_files}
            for future in as_completed(futures):
                try:
                    result = future.result()
                    if result:
                        docs.append(" ".join(result))
                except Exception as e:
                    print(f"Erreur lors du traitement d'un fichier: {e}")
        gc.collect()

        if docs:
            # Utilisation du HashingVectorizer pour obtenir une matrice TF du batch
            hv = HashingVectorizer(n_features=2**19, alternate_sign=False, norm=None)
            batch_matrix = hv.transform(docs)
            # Enregistrement du batch sur disque
            batch_file = os.path.join("../data/npz_matrices", f"tf_batch_{batch_index}.npz")
            save_npz(batch_file, batch_matrix)
            partial_files.append(batch_file)
            print(f"Batch {batch_index} enregistré : forme {batch_matrix.shape}")
            batch_index += 1

    # Si au moins un batch a été traité, on combine toutes les matrices
    if partial_files:
        matrices = [load_npz(f) for f in partial_files]
        full_tf_matrix = vstack(matrices)
        transformer = TfidfTransformer()
        tfidf_matrix = transformer.fit_transform(full_tf_matrix)
        final_file = os.path.join("../data/npz_matrices", "tfidf_matrix.npz")
        save_npz(final_file, tfidf_matrix)
        print("Matrice TF-IDF finale de forme :", tfidf_matrix.shape)
    else:
        print("Aucun document traité.")

if __name__ == '__main__':
    directory = "../data/digraphs"
    process_graphs_in_directory(directory, batch_size=100)

Traitement par batch:   0%|          | 0/1 [00:00<?, ?it/s]

Fichier 0ad1f7ed3d2ed81ce7e68ce0ccb249399e5277bc094f7bc056941789f57ceb6a.json: 180 nœuds, 208 arêtes.
Fichier 0a7f54525b7baab792fbf9160bc27f3b6424b955dae0b50248d14e41e362dbc0.json: 2721 nœuds, 3012 arêtes.
Fichier 0a49ffc1218b6ecfc0a03d115e24fda92e74a09f2bb0ac0d3b472068aa7eb3d4.json: 1763 nœuds, 2749 arêtes.
Fichier 0a7b1cd9ddae54ef50882b41754fbcba30361dd318601dca137bbebf635e8612.json: 1353 nœuds, 1468 arêtes.
Fichier 0a9e739910578192e2ad3f8f805a863d5cbcacbc38cde6488d7bbef5395dba87.json: 2822 nœuds, 4833 arêtes.
Fichier 00a20a5799aa786e2711b43682cf9d533719d5e404c2c5ef1709ac8fdec585f1.json: 443 nœuds, 508 arêtes.
Fichier 0a40f8bb1e3f7ee5ebdbd771368781831260cfbfabf2170f225e2383dc1879ca.json: 2769 nœuds, 3218 arêtes.
Fichier 0a299158b4d10481ba08218ce22bbc08bb78342ab71290ed76667aa93435c9f6.json: 679 nœuds, 1049 arêtes.
Fichier 0a5d1450c71befec5a5aeacaffc6dd7d6514de1693071af6c0b7ed1e2f9f540f.json: 1761 nœuds, 2799 arêtes.
Fichier 0ac7a3df8ddc6e6b5670c1748dfdd80967131f511aaecaeef1c689efe2695

Traitement par batch: 100%|██████████| 1/1 [00:15<00:00, 15.15s/it]

Batch 0 enregistré : forme (50, 524288)


Matrice TF-IDF finale de forme : (50, 524288)


In [5]:
import os
import numpy as np
import tensorflow as tf
from scipy.sparse import load_npz, issparse
from sklearn.decomposition import TruncatedSVD
from tensorflow.keras.models import load_model

# -------------------------
# Fonction de focal loss (pour Keras)
# -------------------------
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-7
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return focal_loss_fixed

# -------------------------
# Chargement du modèle MLP sauvegardé
# -------------------------
keras_model_path = "modele_mlp.h5"
modele = load_model(keras_model_path, custom_objects={'focal_loss_fixed': focal_loss(gamma=2., alpha=0.25)})
print("Modèle Keras chargé.")

# -------------------------
# Chargement de la matrice TF-IDF à partir du dossier test
# -------------------------
tfidf_path = os.path.join("test", "tfidf_matrix.npz")
try:
    X_tfidf = load_npz(tfidf_path)
    print("Matrice TF-IDF chargée de forme :", X_tfidf.shape)
except Exception as e:
    print("Erreur lors du chargement de la matrice TF-IDF :", e)
    exit()

if not issparse(X_tfidf):
    print("Conversion en matrice creuse...")
    X_tfidf = csr_matrix(X_tfidf)

# -------------------------
# Réduction de dimension avec TruncatedSVD
# -------------------------
variance_threshold = 0.90  # pourcentage de variance à capturer
n_components_initial = min(300, X_tfidf.shape[1])
svd_temp = TruncatedSVD(n_components=n_components_initial)
X_temp = svd_temp.fit_transform(X_tfidf)
explained_variance = np.cumsum(svd_temp.explained_variance_ratio_)
n_components = np.argmax(explained_variance >= variance_threshold) + 1
print(f"{n_components} composantes capturent au moins {variance_threshold*100:.0f}% de la variance.")

svd = TruncatedSVD(n_components=n_components)
X_reduced = svd.fit_transform(X_tfidf)
print("Matrice réduite de forme :", X_reduced.shape)

# -------------------------
# Prédiction avec le modèle MLP
# -------------------------
predictions = modele.predict(X_reduced)
predictions_binary = (predictions > 0.5).astype("int32")
print("Prédictions sur la matrice test :")
print(predictions_binary)




FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'modele_mlp.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Fichier .csv avec les prédictions

In [6]:
import pandas as pd

def save_predictions_to_csv(predictions, output_file='predictions.csv'):
    """
    Sauvegarde les résultats des prédictions dans un fichier CSV.
    
    :param predictions: dictionnaire {nom_fichier: prédiction (tableau ou liste)}
    :param output_file: chemin du fichier CSV de sortie
    """
    # Créer un DataFrame à partir du dictionnaire
    df = pd.DataFrame.from_dict(predictions, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'filename'}, inplace=True)
    
    # Sauvegarder le DataFrame en CSV
    df.to_csv(output_file, index=False)
    print(f"Résultats sauvegardés dans {output_file}")

# Exemple d'utilisation avec votre pipeline de prédiction
if __name__ == "__main__":
    test_directory = "test"
    
    # Charger votre modèle Keras sauvegardé
    from tensorflow.keras.models import load_model
    model = load_model("modele_mlp.h5")
    
    # Charger (ou recréer) le transformer TF-IDF utilisé lors de l'entraînement.
    # Ici, nous créons un nouvel objet TfidfTransformer à titre d'exemple.
    from sklearn.feature_extraction.text import TfidfTransformer
    tfidf_transformer = TfidfTransformer()
    
    # Prédire sur les fichiers JSON de test
    predictions = predict_from_json(test_directory, model, tfidf_transformer, svd_model=None, feature_selector=None)
    
    if predictions:
        save_predictions_to_csv(predictions, output_file='predictions.csv')


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'modele_mlp.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)